# Somite Multiomics: miRNA and mRNA Peridogram Matching

We have miRNA and mRNA data from a timecourse experiment that's been performed twice, once with 12.5 minute intervals, and again with 6.25 minute intervals. The 12.5 minute miRNA was sequenced twice. So we have five data sets:

- Sub_0745_I: 12.5 min miRNA, 1st sequencing. 
- Sub_0745_II: 12.5 min miRNA, 2nd sequencing. 
- Sub_0743: 12.5 min mRNA
- Sub_0800: 6.25 min mRNA
- Sub_0814: 6.25 min miRNA

This study was motivated by examing the miRNA hsa-miR-10a-5p, which has cycling in both time courses. What is worrying is that it appears to cycle strongly every 8 time points, even though the intervals are different!

In [ ]:
%load_ext autoreload
%autoreload 2
import somite_study_library as lib

In [ ]:
(sub_0745_I, sub_0745_II, sub_0743, sub_0800, sub_0814) = lib.load_top_expressed_preprocessed_data()
fig = lib.plt.figure(figsize=(15,10))
ax = fig.add_subplot(111)
miRNA_of_interest = 'hsa-miR-10a-5p'
#figsize=(15,20)

for miRNA_set, label in zip((sub_0745_I, sub_0745_II, sub_0814), ('sub_0745_I','sub_0745_II', 'sub_0814')):
    position = list(miRNA_set.var.index).index(miRNA_of_interest)
    y = miRNA_set.X[:,position]
    x = miRNA_set.obs.time
    ax.plot(x,y/lib.np.linalg.norm(y), label=label, linewidth=2.5,marker='o')
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_xlabel('Time (minutes)')
ax.set_ylabel('Normalized expression')
lib.plt.title("hsa-miR-10a-5p Under Three Time Courses")

## Study Strategy

To determine whether the cycling behavior is technical or biological, we will find miRNA-gene pairs in matching timecourses which cycle similarly. We will then take the intersection of these pairs, as we are interested in the ones common to each timecourse. To find these, we will use a different distance metric, using [periodograms](https://en.wikipedia.org/wiki/Periodogram). 

Because mRNA behavior lags behind the miRNA changes, periodograms are useful because they only measure the relative power of different frequencies, not the phase. 




In [ ]:
# We are looking for mRNA genes which cycle similarly to a target miRNA. 
# Our strategy is to calculate the periodogram for the target miRNA, and all genes. 
# Sort by distance, and return n closest. 
# We'll look out 3 pairs of data. 

n_closest = 75
s800_closest_genes = lib.get_mutual_periodgram_genes(
    adata_miRNA=sub_0814, 
    target_miRNA='hsa-miR-10a-5p', 
    adata_mRNA=sub_0800,
    n_closest=n_closest
)

s743_1_closest_genes = lib.get_mutual_periodgram_genes(
    adata_miRNA=sub_0745_I, 
    target_miRNA='hsa-miR-10a-5p', 
    adata_mRNA=sub_0743,
    n_closest=n_closest
)

s743_2_closest_genes = lib.get_mutual_periodgram_genes(
    adata_miRNA=sub_0745_II, 
    target_miRNA='hsa-miR-10a-5p', 
    adata_mRNA=sub_0743,
    n_closest=n_closest
)

common_genes = (
    set(s800_closest_genes.index) 
    & set(s743_1_closest_genes.index)
    & set(s743_2_closest_genes.index)
)

print(f'The Common Genes are: {", ".join(common_genes)}')

We are not sure about the significance of this finding. We visited [miRDB](http://mirdb.org/), an online database for miRNA target prediction, which lists RPL13A as a target, but not the other three. 


We Also print out the expression circles of the four shared close mRNAs with the target miRNA in one pic for better viewing about the oscillation pattern in three groups of difference samples and difference timecourses.

In [ ]:
lib.plot_combine_targetMiR_ComMR(common_genes,sub_0814,sub_0800)
lib.plt.title("800 & 814")
lib.plot_combine_targetMiR_ComMR(common_genes,sub_0745_I,sub_0743)
lib.plt.title("0743 & 0745_I")
lib.plot_combine_targetMiR_ComMR(common_genes,sub_0745_II,sub_0743)
lib.plt.title("0743 & 0745_II")

TUBA1A shares very similar period as our target miRNA in all three plots

## Background and Future Work

### K-means clustering in periodogram space.

One concern we have is how noisy the data is, as well as the possible 8 cycle sequencing artifact. So we would like to survey more miRNA species. One way to do this is to cluster the top expressed miRNA into groups, and see if they all share the same artifact.

In [ ]:
print("0745_I, hsa-miR-10a-5p group")
K = 6
s745I_pd_array = lib.get_periodgram(sub_0745_I)
C1, centroids = lib.k_means_from_pd_data(s745I_pd_array,K=K)
lib.plot_targetmiRNA_group(sub_0745_I,C1)


In [ ]:
print("0745_II, hsa-miR-10a-5p group")
K = 6
s745II_pd_array = lib.get_periodgram(sub_0745_II)
C2, centroids = lib.k_means_from_pd_data(s745II_pd_array,K=K)
lib.plot_targetmiRNA_group(sub_0745_II,C2)

In [ ]:
print("0814, hsa-miR-10a-5p group")
K = 6
s814_pd_array = lib.get_periodgram(sub_0814)
C3, centroids = lib.k_means_from_pd_data(s814_pd_array,K=K)
lib.plot_targetmiRNA_group(sub_0814,C3)

We trying to find the common genes shared in these three groups of miRNAs and plot out their expression circle for comparison between sub_814, sub_745_I and sub_745_II. However, since the K means algorithm is ML, it trains itself each time and we got different group each run. 